In [ ]:
import numpy as np
import pymc as pm
from matplotlib import pyplot as plt

class Bandits(object):

    """
    This class represents N bandits machines.

    parameters:
        p_array: a (n,) Numpy array of probabilities >0, <1.

    methods:
        pull( i ): return the results, 0 or 1, of pulling 
                   the ith bandit.
    """

    def __init__(self, p_array):
        self.p = p_array
        self.optimal = np.argmax(p_array)

    def pull(self, i):
        # i is which arm to pull
        return np.random.rand() < self.p[i]

    def __len__(self):
        return len(self.p)

class BayesianStrategy(object):

    """
    Implements a online, learning strategy to solve
    the Multi-Armed Bandit problem.
    
    parameters:
        bandits: a Bandit class with .pull method
    
    methods:
        sample_bandits(n): sample and train on n pulls.

    attributes:
        N: the cumulative number of samples
        choices: the historical choices as a (N,) array
        bb_score: the historical score as a (N,) array
    """

    def __init__(self, bandits):

        self.bandits = bandits
        n_bandits = len(self.bandits)
        self.wins = np.zeros(n_bandits)
        self.trials = np.zeros(n_bandits)
        self.N = 0
        self.choices = []
        self.bb_score = []

    def sample_bandits(self, n=1):

        bb_score = np.zeros(n)
        choices = np.zeros(n)

        P0_samples = [np.random.rand()]
        P1_samples = [np.random.rand()]
        P2_samples = [np.random.rand()]

        for k in range(n):
            # sample from the bandits's priors, and select the largest sample

            choice = np.argmax([np.random.choice(P0_samples), np.random.choice(P1_samples), np.random.choice(P2_samples)])

            print();
            print();
            print(k, choice);

            # sample the chosen bandit
            result = self.bandits.pull(choice)

            # update priors and score
            self.wins[choice] += result
            self.trials[choice] += 1
            bb_score[k] = result
            self.N += 1
            choices[k] = choice

            if choice == 0:
                P0 = pm.Uniform('P0', 0, 1)
                X0 = pm.Binomial('X0', value = self.wins[0], n = self.trials[0], p = P0, observed = True)
                mcmc0 = pm.MCMC([P0, X0])
                mcmc0.sample(15000, 5000)
                P0_samples = mcmc0.trace('P0')[:]
            elif choice == 1:
                P1 = pm.Uniform('P1', 0, 1)
                X1 = pm.Binomial('X1', value = self.wins[1], n = self.trials[1], p = P1, observed = True)
                mcmc1 = pm.MCMC([P1, X1])
                mcmc1.sample(15000, 5000)
                P1_samples = mcmc1.trace('P1')[:]
            else:
                P2 = pm.Uniform('P2', 0, 1)
                X2 = pm.Binomial('X2', value = self.wins[2], n = self.trials[2], p = P2, observed = True)
                mcmc2 = pm.MCMC([P2, X2])
                mcmc2.sample(15000, 5000)
                P2_samples = mcmc2.trace('P2')[:]


        self.bb_score = np.r_[self.bb_score, bb_score]
        self.choices = np.r_[self.choices, choices]
        return

hidden_prob = np.array([0.85, 0.60, 0.75])
bandits = Bandits(hidden_prob)

bayesian_strat = BayesianStrategy(bandits)

bayesian_strat.sample_bandits(10000)

def regret(probabilities, choices):
    w_opt = probabilities.max()
    return (w_opt - probabilities[choices.astype(int)]).cumsum()

_regret = regret(hidden_prob, bayesian_strat.choices)
plt.plot(_regret, lw=3)

plt.title("Total Regret of Bayesian Bandits Strategy (MCMC)")
plt.xlabel("Number of pulls")
plt.ylabel("Regret after $n$ pulls")
plt.show()




0 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

1 2
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

2 2
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

3 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

4 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

6 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

7 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

8 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

9 2
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

10 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

11 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

12 0
 [-----

 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

99 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

100 2
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

101 2
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

102 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.5 sec

103 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

104 2
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

105 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.5 sec

106 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

107 2
 [-----------------100%-----------------] 15000 of 15000 complete in 0.5 sec

108 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.9 sec

109 2
 [-----------------100%-----------------] 15000 of 15000 complete in 0.6 sec


 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

196 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

197 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

198 2
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

199 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

200 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

201 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

202 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

203 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

204 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

205 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

206 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

293 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

294 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

295 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

296 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

297 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

298 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.5 sec

299 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.5 sec

300 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.5 sec

301 2
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

302 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.5 sec

303 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

390 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

391 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

392 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

393 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

394 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

395 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

396 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

397 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

398 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

399 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

400 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

...

 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5580 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5581 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5582 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5583 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5584 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5585 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5586 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5587 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5588 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5589 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5590 0
 [-----------------100%-----------------] 15000 of 15000 complete

 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5676 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5677 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5678 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5679 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5680 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5681 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5682 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5683 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5684 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5685 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.5 sec

5686 0
 [-----------------100%-----------------] 15000 of 15000 complete

 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5772 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5773 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5774 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5775 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5776 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5777 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5778 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.5 sec

5779 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.5 sec

5780 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.5 sec

5781 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.6 sec

5782 0
 [-----------------100%-----------------] 15000 of 15000 complete

 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5868 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5869 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5870 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5871 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5872 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5873 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5874 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5875 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5876 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5877 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5878 0
 [-----------------100%-----------------] 15000 of 15000 complete

 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5963 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5964 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5965 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.5 sec

5966 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.5 sec

5967 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5968 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5969 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5970 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5971 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5972 0
 [-----------------100%-----------------] 15000 of 15000 complete in 0.4 sec

5973 0
 [-----------------100%-----------------] 15000 of 15000 complete